# 1. Recipricol Rank


In [21]:
# read the prediction results dict
import json

def read_json(file):
    with open(file, 'r') as f:
        return json.load(f)

results_dict = read_json('./eval_results/eval_051524_173722/predictions.json')

In [34]:
for i in range(len(results_dict)):
    print('raw_input: ')
    print(results_dict[str(i)]['raw_input'])
    print('*************************************************')
    print('input: ')
    print(results_dict[str(i)]['input'])
    print('*************************************************')
    print('ground_truth: ')
    print(results_dict[str(i)]['ground_truth'])
    print('*************************************************')
    print('generated_responses: ')
    print(results_dict[str(i)]['generated_responses'])
    print('*************************************************')
    break

raw_input: 
Below is the previous historical purchases and reviews of the user:
```
Item title: Abduction 
 Item description: HD. Searching for his true identity, a teen is hunted by assassins after finding a picture of himself as a kid on a missing-persons website. 
 rating: 4.0 
 review: The movie had lots of unexpected twists and a great adventure, lots of action scenes.  The ending is very unpredictable.-------
```
And here is the user's intention: Hey, I'm in the mood for a captivating romantic drama with a unique twist, any recommendations?
Please infer the user's preference based on historical purchases and reviews along with the user's intention, and then recommend an item for this user. Please just give the title of the recommended item.
*************************************************
input: 
Below is the previous historical purchases and reviews of the user:
```
Item title: Abduction 
 Item description: HD. Searching for his true identity, a teen is hunted by assassins afte

In [32]:
# calculate the reciprocal rank
reciprocal_rank_list = []
for i in range(len(results_dict)):
    reciprocal_rank = 0
    ground_truth = results_dict[str(i)]['ground_truth']
    if ground_truth.startswith("Title"):
        ground_truth = ground_truth.split("Description:")[0].strip().strip("\n").strip("Title: ")
    generated_responses = results_dict[str(i)]['generated_responses']
    for j in range(len(generated_responses)):
        if ground_truth == generated_responses[j]['sequence']:
            reciprocal_rank = 1 / (j + 1)
            break
    reciprocal_rank_list.append(reciprocal_rank)

In [33]:
import numpy as np

np.mean(reciprocal_rank_list)

0.019417475728155338

# 2. Category match rate metrics

In [5]:
# construct training data for the model
# import json

# # Function to read a .jsonl file
# def read_jsonl(file_path):
#     data = []
#     with open(file_path, 'r') as file:
#         for line in file:
#             # Parse each line as JSON and append to the data list
#             data.append(json.loads(line))
#             # break
#     return data

# movies_meta_data = read_jsonl('./data/meta_Movies_and_TV.jsonl')

# use pandas dataframe to load the meta data
import pandas as pd

# Read the dataframe
movies_meta_data = pd.read_json('./data/meta_Movies_and_TV.jsonl', lines=True)

# import gc

# # Delete the dataframe
# del movies_meta_data

# # Run garbage collector to free up memory
# gc.collect()

In [6]:
movies_meta_data.iloc[0]

main_category                                            Prime Video
title                                                           Glee
subtitle                                        UnentitledUnentitled
average_rating                                                   4.7
rating_number                                                 2004.0
features                 [IMDb 6.8, 2013, 22 episodes, X-Ray, TV-14]
description        [Entering its fourth season, this year the mem...
price                                                          22.39
images             [{'360w': 'https://images-na.ssl-images-amazon...
videos                                                            []
store                                                           None
categories         [Comedy, Drama, Arts, Entertainment, and Cultu...
details            {'Content advisory': ['Violence', 'substance u...
parent_asin                                               B00ABWKL3I
bought_together                   

In [7]:
def precision_recall_f1(ground_truth_categories, recommended_categories):
    match_count = len(set(ground_truth_categories).intersection(set(recommended_categories)))
    precision = match_count / len(recommended_categories) if recommended_categories else 0
    recall = match_count / len(ground_truth_categories) if ground_truth_categories else 0

    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score

def jaccard_similarity(categories1, categories2):
    intersection = len(set(categories1).intersection(set(categories2)))
    union = len(set(categories1).union(set(categories2)))

    return intersection / union if union != 0 else 0

In [20]:
# calculate category match rate metrics
f1_list = []
jaccard_similarity_list = []
for i in range(len(results_dict)):
    # get the ground_truth movie title
    ground_truth = results_dict[str(i)]['ground_truth']
    if ground_truth.startswith("Title"):
        ground_truth = ground_truth.split("Description:")[0].strip().strip("\n")[6:].strip()
    ground_truth_categories = movies_meta_data[movies_meta_data['title'] == ground_truth]['categories'].tolist()[0]
    generated_responses = results_dict[str(i)]['generated_responses']
    recommended_categories_metrics_f1 = []
    recommended_categories_metrics_jaccard = []
    for j in range(len(generated_responses)):
        recommended_title = generated_responses[j]['sequence']
        # try:
        recommended_categories = movies_meta_data[movies_meta_data['title'] == recommended_title]['categories'].tolist()[0]
        # except IndexError:
        #     print(recommended_title)
        #     print(movies_meta_data[movies_meta_data['title'] == recommended_title]['categories'])
        #     continue
        # print(recommended_categories
        recommended_categories_metrics_f1.append(precision_recall_f1(ground_truth_categories, recommended_categories))
        recommended_categories_metrics_jaccard.append(jaccard_similarity(ground_truth_categories, recommended_categories))
    f1_list.append(recommended_categories_metrics_f1)
    jaccard_similarity_list.append(recommended_categories_metrics_jaccard)

In [21]:
f1_list[0][0][2], max(jaccard_similarity_list[0])

(1.0, 1.0)

In [22]:
f1_list_test = []
for i in range(5):
    f1_list_test.append(f1_list[0][i][2])

In [23]:
f1_list_test

[1.0, 0.5, 0, 1.0, 0.5]

In [25]:
import numpy as np

f1_max_list = []
jaccard_similarity_max_list = []
for i in range(len(results_dict)):
    f1_list_for_one_result = []
    jaccard_similarity_list_for_one_result = []
    for j in range(len(generated_responses)):
        f1_list_for_one_result.append(f1_list[i][j][2])
        jaccard_similarity_list_for_one_result.append(jaccard_similarity_list[i][j])
        
    # take max of each list
    f1_max_list.append(max(f1_list_for_one_result))
    jaccard_similarity_max_list.append(max(jaccard_similarity_list_for_one_result))
    
np.mean(f1_max_list), np.mean(jaccard_similarity_max_list)

(0.5980813684697179, 0.5042533518261674)

# 3. If all recommendation is random selection. What is the expected value of the metrics?

In [1]:
import json
# asins_small.json to be used for meta data matching
json_file_asins = './data/asins_small.json'
# json_file_asins = './data/meta_asins.json'
with open(json_file_asins, "r") as file:
    asin_dict = json.load(file)
    
len(asin_dict)

29205

In [14]:
import random
def get_random_recommendation(num_of_recommendation, asin_dict):
    random_asin_list = random.sample(list(asin_dict.keys()), num_of_recommendation)
    random_movie_list = []
    for asin in random_asin_list:
        random_movie_list.append(asin_dict[asin][0])
    return random_movie_list

In [19]:
get_random_recommendation(5, asin_dict)

['King Kong',
 '300 [Blu-ray]',
 'Veronica Mars: Season 3',
 'U.S. Marshals (Special Edition)',
 'Thunder Bay [VHS]']

In [36]:
# calculate the reciprocal rank
reciprocal_rank_list = []
for i in range(len(results_dict)):
    reciprocal_rank = 0
    ground_truth = results_dict[str(i)]['ground_truth']
    if ground_truth.startswith("Title"):
        ground_truth = ground_truth.split("Description:")[0].strip().strip("\n").strip("Title: ")
    generated_responses = get_random_recommendation(5, asin_dict)
    for j in range(len(generated_responses)):
        if ground_truth == generated_responses[j]:
            reciprocal_rank = 1 / (j + 1)
            break
    
    reciprocal_rank_list.append(reciprocal_rank)

In [37]:
import numpy as np

np.sum(reciprocal_rank_list),  \
np.mean(reciprocal_rank_list)

(0, 0.0)